# Exemple d'utilisation du package

## Importation du package et Lecture du fichier de données

In [1]:
# import nécessaire au fonctionnement du package
from mypackage import Strategy_Manager, Strategy, strategy, DataFileReader

# import supplémentaire pour les stratégies utilisées comme exemple
import pandas as pd
import warnings
# Désactiver tous les avertissements
warnings.filterwarnings('ignore')

In [2]:
# Chemin vers le fichier de données sous format .csv ou .parquet
filepath = 'fichier_donnée.csv' 
# filepath = 'data.parquet'
# filepath = 'data2.parquet'

# Initialiser le lecteur de fichiers en indiquant le format des dates
reader = DataFileReader(date_format='%d/%m/%Y') 

# Lecture du fichier
data = reader.read_file(filepath, date_column='Date_Price')
data

,AAVEUSDT,ACMUSDT,ADAUSDT,ADXUSDT,AGLDUSDT,ALGOUSDT,ALICEUSDT,ALPACAUSDT,ALPHAUSDT,AMPUSDT
Date_Price,,,,,,,,,,
2022-01-01,266.4,5.211,1.3800,0.5729,1.705,1.7414,13.380,0.6264,0.6995,0.04964
2022-01-02,265.3,5.195,1.3770,0.5799,1.828,1.6734,13.060,0.6085,0.6978,0.04897
2022-01-03,267.2,5.152,1.3190,0.5798,1.718,1.7949,12.640,0.6186,0.7019,0.05239
2022-01-04,252.1,5.107,1.3090,0.5613,1.637,1.6725,13.650,0.6163,0.6981,0.04929
2022-01-05,230.3,4.727,1.2300,0.5276,1.570,1.5554,12.230,0.5660,0.6276,0.04682
...,...,...,...,...,...,...,...,...,...,...
2022-12-26,56.0,2.346,0.2652,0.1212,0.270,0.1715,1.091,0.2529,0.0801,0.00318
2022-12-27,57.5,2.345,0.2603,0.1182,0.264,0.1701,1.093,0.2504,0.0786,0.00312
2022-12-28,53.8,2.288,0.2482,0.1143,0.259,0.1670,1.058,0.2477,0.0762,0.00309


## Création d'une stratégie d'exemple

In [3]:
# Création d'une stratégie par héritage
class MovingAverageCrossover(Strategy):
    """Stratégie de croisement de moyennes mobiles pour plusieurs actifs."""
    
    def __init__(self, assets, short_window=20, long_window=50, rebalancing_frequency='D', 
                 allocation_method='equal'):
        super().__init__(rebalancing_frequency=rebalancing_frequency, assets=assets)
        self.short_window = short_window
        self.long_window = long_window
        self.allocation_method = allocation_method

    def get_position(self, historical_data, current_position):
        if len(historical_data) < self.long_window:
            return {asset: 0 for asset in self.assets}
        
        signals = {}
        for asset in self.assets:
            short_ma = historical_data[asset].rolling(self.short_window).mean()
            long_ma = historical_data[asset].rolling(self.long_window).mean()
            signals[asset] = 1 if short_ma.iloc[-1] > long_ma.iloc[-1] else -1
        
        # Allocation des positions selon la méthode choisie
        if self.allocation_method == 'equal':
            position_size = 1.0 / len(self.assets)
            positions = {asset: signal * position_size for asset, signal in signals.items()}
        else:  # 'signal_weighted'
            total_signals = sum(abs(signal) for signal in signals.values())
            positions = {asset: signal / total_signals for asset, signal in signals.items()}
            
        return positions

## Mise en place des variables nécessaire aux backtests

In [4]:
# Création de la liste des actifs utilisé
all_asset = data.columns.to_list()

# Création des instances et exécution des backtests
ma_strat_default = MovingAverageCrossover(assets=all_asset, short_window=20, long_window=50)
ma_strat_weekly = MovingAverageCrossover(assets=all_asset, short_window=20, long_window=50, rebalancing_frequency='W')     # Weekly rebalancing
ma_strat_monthly = MovingAverageCrossover(assets=all_asset, short_window=20, long_window=50, rebalancing_frequency='M')    # Monthly rebalancing

# Création des dictionnaires de coûts
transaction_cost = {'AAVEUSDT': 0.5, 'ACMUSDT': 0.2, 'ADAUSDT': 0.4, 'ADXUSDT': 0.6, 'AGLDUSDT': 0.7, 
                    'ALGOUSDT': 0.001, 'ALICEUSDT': 0.8, 'ALPACAUSDT': 2, 'ALPHAUSDT': 0.8, 'AMPUSDT': 0.051}
slippage_cost = {'AAVEUSDT': 0.53, 'ACMUSDT': 0.03, 'ADAUSDT': 0.33, 'ADXUSDT': 0.63, 'AGLDUSDT': 0.73, 
                 'ALGOUSDT': 0.001, 'ALICEUSDT': 0.8, 'ALPACAUSDT': 21, 'ALPHAUSDT': 0.8, 'AMPUSDT': 0.01}

# Création d'un dictionnaire partiel de coûts 
transaction_cost_partiel = {'AAVEUSDT': 0.5, 'ACMUSDT': 0.2,}

# Création d'un dictionnaire des différentes stratégies à comparer avec 
#   en clé, le nom choisi de la stratégie
#   en valeur, un tuple contenant la stratégie, les coûts de transactions, les coûts de slippage
dico_strat = {
    'ma_strat_default': (ma_strat_default, 0.1, 0.2),
    'ma_strat_weekly': (ma_strat_weekly, transaction_cost_partiel, None),
    'ma_strat_monthly': (ma_strat_monthly, transaction_cost, slippage_cost),
}

# Création de l'instance utile à la comparaison
manager = Strategy_Manager(data, dico_strat)

# Exécution des backtests
manager.run_backtests()

## Résultat du Backtest

In [5]:
# Affichage des statistiques pour chaque stratégie
manager.print_statistics()


Statistiques : 
                  total_return  annual_return  profit_factor  volatility  \
ma_strat_default       -0.3193        -0.2338         0.4892      0.0745   
ma_strat_weekly        -0.0694        -0.2895         0.3830      0.0704   
ma_strat_monthly       -0.9939        -1.0000         0.0002      4.0918   

                  sharpe_ratio  max_drawdown  sortino_ratio  VaR_95%  \
ma_strat_default       -3.1364       -0.3280        -2.5215  -0.0091   
ma_strat_weekly        -4.1098       -0.0732        -2.8915  -0.0102   
ma_strat_monthly       -0.2444       -0.9939        -0.1401  -0.6143   

                  CVaR_95%  Profit/Loss_Ratio  num_trades  win_rate  
ma_strat_default   -0.0129             0.6194          72    0.4413  
ma_strat_weekly    -0.0123             0.4377          69    0.4667  
ma_strat_monthly   -0.6179             0.0015          47    0.1000  


In [6]:
# Affichage des statistiques pour une stratégie précise
manager.print_statistics(strategy_name="ma_strat_default")


Statistiques de la strategie 'ma_strat_default':
                  total_return  annual_return  profit_factor  volatility  \
ma_strat_default       -0.3193        -0.2338         0.4892      0.0745   

                  sharpe_ratio  max_drawdown  sortino_ratio  VaR_95%  \
ma_strat_default       -3.1364        -0.328        -2.5215  -0.0091   

                  CVaR_95%  Profit/Loss_Ratio  num_trades  win_rate  
ma_strat_default   -0.0129             0.6194          72    0.4413  


In [7]:
# Affichage des statistiques des différents actifs pour la stratégie précise
manager.print_statistics(strategy_name="ma_strat_default",detail=True)

# Affichage des statistiques des différents actifs pour toutes les stratégies 
# manager.print_statistics(detail=True)


Statistiques : ma_strat_default
            total_return  annual_return  profit_factor  volatility  \
AAVEUSDT         -0.3741        -0.2771         0.6051      0.1639   
ACMUSDT          -0.2397        -0.1728         0.7308      0.1594   
ADAUSDT          -0.2279        -0.1640         0.6884      0.1247   
ADXUSDT          -0.4092        -0.3053         0.5075      0.1619   
AGLDUSDT         -0.5435        -0.4189         0.4843      0.2000   
ALGOUSDT         -0.4234        -0.3169         0.5136      0.1599   
ALICEUSDT        -0.2293        -0.1650         0.7281      0.1364   
ALPACAUSDT       -0.3570        -0.2634         0.5485      0.1551   
ALPHAUSDT        -0.3555        -0.2622         0.6137      0.1505   
AMPUSDT           0.0242         0.0167         1.0617      0.0933   

            sharpe_ratio  max_drawdown  sortino_ratio  VaR_95%  CVaR_95%  \
AAVEUSDT         -1.6909       -0.3871        -1.1968  -0.0121   -0.0319   
ACMUSDT          -1.0841       -0.2766      

### Analyse Backtest

Analyse.................

## Graphique

In [24]:
# Choix du backend utilisé pour les graphiques
backend = 'plotly' # 'plotly' # 'matplotlib' # 'seaborn'

In [25]:
# Graphique des différentes stratégies avec possibilité d'inclusion de coûts 
manager.plot_all_strategies(backend=backend,include_costs=True)

In [26]:
# Graphique des différents actifs d'une stratégie précise avec possibilité d'inclusion de coûts 
manager.plot_strategy(strategy_name="ma_strat_default",backend=backend,include_costs=True)

In [27]:
# Histogramme de comparaison des différentes stratégies
manager.compare_strategies(backend=backend)

### Analyse Graphique

Analyse..........